# ALgoritmo que transforma os valores em matrizes

In [14]:
import ast
from textblob import TextBlob
import json

In [15]:
#Diretorios dos arquivos de similaridade
similaridades = ['similaridades-camera.json','similaridades-cells.json','similaridades-dvds.json','similaridades-laptops.json','similaridades-routers.json']

#Diretorios dos arquivos de contexto
contextos = ['contexto-camera.json','contexto-cells.json','contexto-dvds.json','contexto-laptops.json','contexto-routers.json']

#Diretorios dos arquivos de contextualizacao
contextualizacao = ['contextualizacao-camera.json','contextualizacao-cells.json','contextualizacao-dvds.json','contextualizacao-laptops.json','contextualizacao-routers.json']

palavras = ['palavras-camera.json','palavras-cells.json','palavras-dvds.json','palavras-laptops.json','palavras-routers.json']

words = {}

caminhosG =  ['similaridades-camera.json','similaridades-cells.json','similaridades-dvds.json','similaridades-laptops.json','similaridades-routers.json']

caminhosT = ['contexto-camera.json','contexto-cells.json','contexto-dvds.json','contexto-laptops.json','contexto-routers.json']

caminhosC = ['termo-documento-camera.json','termo-documento-cells.json','termo-documento-dvds.json', 'termo-documento-laptops.json','termo-documento-routers.json']

In [17]:
for index, palavra in enumerate(palavras):
    # obtendo palavras
    with open('../../datasets_processed/' + palavras[index], 'r') as fp:
        data_words = json.load(fp)
    
    matrizG = [[1.0]*len(data_words.keys()) for i in range(len(data_words.keys()))]
    matrizT = [[1.0]*len(data_words.keys()) for i in range(len(data_words.keys()))]
    matrizC = []
    
    # obtendo similaridades do spacy
    with open('../../datasets_processed/similaridades/' + similaridades[index], 'r') as fp:
        data_similaridades = json.load(fp)
        
    for i, value in enumerate(data_similaridades):
        for j, value_simi in enumerate(data_similaridades[value]):
            print()
            matrizG[i][j] = data_similaridades[value][value_simi]
            matrizG[j][i] = data_similaridades[value][value_simi]
            
    # obte o NPMI das palavras
    with open('../../datasets_processed/contextos/' + contextos[index], 'r') as fp:
        data_contextos = json.load(fp)
        
    for i, value in enumerate(data_contextos):
        for j, value_context in enumerate(data_contextos[value]):
            matrizT[i][j] = data_contextos[value][value_context]
            matrizT[j][i] = data_contextos[value][value_context]
            
    # obtendo os mapeamentos de term-document
    with open('../../datasets_processed/contextualizacao/' + contextualizacao[index], 'r') as fp:
        data_contextualizacao = json.load(fp)
        
    for key, values in data_contextualizacao.items():
        docs = []
        
        for value in values:
            if(len(value) == 0):
                # se o documento for vazio significa que o atributo não aparece neste documento
                docs.append(0.0)
            else:
                tam_word = len(TextBlob(key).words) # tamanho da palavra
                qnt_freq = 0 # variavel que guarda a frequencia da palavra no documento
                qnt_all = 0 # variavel que guarda a quantidade total de palavras no documento
                for sent in value:
                    tam = len(TextBlob(sent.lower()).words) #Quantidade de palavras na sentença
                    if(' ' in key):
                        qnt_freq += sent.lower().count(key) #Frequencia da palavra na sentença
                        qnt_all += abs(tam - (tam_word*qnt_freq)) + qnt_freq #Quantidade de palavras total na sentenca
                    else:
                        qnt_freq += TextBlob(sent.lower()).words.count(key)
                        qnt_all += len(TextBlob(sent.lower()).words)
                        
                docs.append(qnt_freq/qnt_all)
    matrizC.append(docs)
    
    # salvando matriz de similaridades
    with open('../../datasets_processed/matrizes/' + caminhosG[index], 'w') as fp:
        data_saved_simi = {}
        for i, g in enumerate(matrizG):
            data_saved_simi[i] = g
        json.dump(data_saved_simi, fp)
            
    # salvando matriz de contexto
    with open('../../datasets_processed/matrizes/' + caminhosT[index], 'w') as fp:
        data_saved_ctx = {}
        for i, t in enumerate(matrizT):
            data_saved_ctx[i] = t
        json.dump(data_saved_ctx, fp)
        
    # salvando matriz de contextualização
    with open('../../datasets_processed/matrizes/' + caminhosC[index], 'w') as fp:
        data_saved_cta = {}
        for i, c in enumerate(matrizC):
            data_saved_cta[i] = c
        json.dump(data_saved_cta, fp) 